In [127]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input"))
import math

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#TODO  https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM

# print(tf.__version__)
# fix random seed for reproducibility
np.random.seed(7)

['weatherAUS.csv']


**Load and discover dataset**

In [128]:
mydateparser = lambda x: pd.datetime.strptime(x, "%Y-%m-%d")
df = pd.read_csv("../input/weatherAUS.csv", parse_dates=['Date'], date_parser=mydateparser)
df.head(5)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [ ]:
#df.describe()
df.info()

In [ ]:
#Location
#
print(len(df["Location"].unique()))
df["Location"].unique()
# todo : place them on a map, distance vetween town, distance to ocean (in W / E/ / S / Noth)

In [ ]:
#df.describe('')
#
# MinTemp et MaxTemp
%matplotlib inline
import matplotlib.pyplot as plt
df.hist(bins=50, figsize=(20,15))
plt.show()





In [ ]:
#df.info()

**Jeu d'entrainement et jeu de test**



In [129]:
from sklearn.model_selection import train_test_split
# V1 : Totalement aléatoire.

train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)
train_set.dropna()

# https://www.kaggle.com/jsphyg/weather-dataset-rattle-package/discussion/78316#latest-495878
train_set = train_set.drop('RISK_MM', axis=1)
test_set = test_set.drop('RISK_MM', axis=1)

X_train = train_set


*Suppression champs date/Ville (Plus tard)*
Date => Saison ? Rnn LSTM.
Ville => Ajout de la ville ? Longitude / Latitude. Cas Vent qui vient de ocean vs terre ?




In [130]:
def removeUnusedVariable(df):
    df.drop('Location', axis=1, inplace=True)
    #df.drop('Date', axis=1, inplace=True)
    
removeUnusedVariable(X_train)



In [131]:
# Modification vers numérique
X_train["RainToday"] = X_train["RainToday"].map({'No': 0, 'Yes': 1})
X_train["RainTomorrow"] = X_train["RainTomorrow"].map({'No': 0, 'Yes': 1})
X_train.head(5)





,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
17969,2009-10-14,15.1,23.9,0.0,NaN,NaN,SSW,67.0,NW,W,19.0,22.0,38.0,68.0,1001.9,1002.4,NaN,NaN,19.8,14.3,0.0,0
124769,2011-09-26,9.7,14.2,7.6,NaN,NaN,WSW,50.0,WNW,W,15.0,28.0,91.0,56.0,1008.2,1007.7,NaN,NaN,11.1,13.4,1.0,1
39287,2010-04-20,13.2,25.4,0.0,3.2,8.8,ENE,30.0,W,E,6.0,17.0,79.0,63.0,1025.2,1021.5,6.0,5.0,21.2,24.0,0.0,0
127749,2011-07-12,7.6,14.8,0.0,4.0,7.0,WNW,94.0,WNW,WNW,30.0,35.0,52.0,45.0,1004.6,1001.4,NaN,NaN,11.1,12.9,0.0,0
40651,2015-04-13,12.9,22.2,0.0,4.0,7.9,S,37.0,SW,SSE,15.0,20.0,69.0,52.0,1023.0,1021.2,6.0,2.0,18.8,20.6,0.0,0


*Date*

In [132]:
#import time
# a/ As a number 1-12 is month revelant ? => No
# b/ is a numer 1-6 
import datetime
def processDate(df):
    df['Month'] = df['Date'].dt.month
    #datetime.date(2008, 12, 25)
    #df['SummerDay1'] = df['Date'] - datetime.date(df['Date'].dt.year, 6, 21)
    df['SummerDay'] = df['Month'].map({1: 6, 2:5, 3:4, 4:3, 5:2, 6:1, 7:1, 8:2, 9:3, 10:4, 11:5, 12:6})
    df.drop('Month', axis=1, inplace=True)
    df.drop('Date', axis=1, inplace=True)

    
processDate(X_train)
X_train.head(5)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,SummerDay
17969,15.1,23.9,0.0,NaN,NaN,SSW,67.0,NW,W,19.0,22.0,38.0,68.0,1001.9,1002.4,NaN,NaN,19.8,14.3,0.0,0,4
124769,9.7,14.2,7.6,NaN,NaN,WSW,50.0,WNW,W,15.0,28.0,91.0,56.0,1008.2,1007.7,NaN,NaN,11.1,13.4,1.0,1,3
39287,13.2,25.4,0.0,3.2,8.8,ENE,30.0,W,E,6.0,17.0,79.0,63.0,1025.2,1021.5,6.0,5.0,21.2,24.0,0.0,0,3
127749,7.6,14.8,0.0,4.0,7.0,WNW,94.0,WNW,WNW,30.0,35.0,52.0,45.0,1004.6,1001.4,NaN,NaN,11.1,12.9,0.0,0,1
40651,12.9,22.2,0.0,4.0,7.9,S,37.0,SW,SSE,15.0,20.0,69.0,52.0,1023.0,1021.2,6.0,2.0,18.8,20.6,0.0,0,3


*Wind Direction*

In [ ]:
# return 2 values :  
#   -1 to 1 for West to East
#  - 1 to 1 for North to South
# winDirection : exemple S, NE, NNE (Max : 3 caracter)


#df["WindGustDirCode"] = df["WindGustDir"]
def convertWindDirectionGlobal(winDirection) :
    nbValue = len(winDirection)
    # 6  : 3 * 2 * 1 (nb caractere commun)
    coef = 6 / nbValue
    dirWE = winDirection.count("W") * -1
    dirWE += winDirection.count("E")
    dirWE = (dirWE * coef) / 6
    
    dirNS = winDirection.count("N")* -1
    dirNS += winDirection.count("S")
    dirNS = (dirNS * coef) / 6
    return dirWE, dirNS

def convertWindDirectionAxe(winDirection, neg, pos) :
    nbValue = len(winDirection)
    # 6  : 3 * 2 * 1 (nb caractere commun)
    coef = 6 / nbValue
    dir = winDirection.count(neg) * -1
    dir += winDirection.count(pos)
    dir = (dir * coef) / 6
    
    return dir

        
#from sklearn.preprocessing import FunctionTransformer
#attr_adder = WindAttributesConverter()
#df["WindGustDirCode"] = df["WindGustDir"].str.len()
#df["WindGustDirCode"] = attr_adder.transform(df)
#df.head(5)**
#attr_adder = FunctionTransformer(convertWindDirectionGlobal)

def changeColumnWinDir(df, name):
    # WE => 2 valeurs opposées. Resultat sera 0
    df[name].fillna("WE", inplace=True)
    df[name + "WE"] = df[name].map(lambda x: convertWindDirectionAxe(x, "W", "E") , na_action='ignore' )
    df[name + "NS"] = df[name].map(lambda x: convertWindDirectionAxe(x, "N", "S") , na_action='ignore' )
    df.drop(name, axis=1, inplace=True)
    
def changeAllColumnWinDir(df):
    changeColumnWinDir(df, "WindGustDir")
    changeColumnWinDir(df, "WindDir9am")
    changeColumnWinDir(df, "WindDir3pm")
    #categorical_columns = ['WindGustDir', 'WindDir3pm', 'WindDir9am']
    #df = pd.get_dummies(df, columns=categorical_columns)
    
    return df
X_train = changeAllColumnWinDir(X_train)





In [ ]:
#X_train["WindGustDir"] 
#X_train["WindGustDirNS"] = X_train["WindGustDir"]
#X_train["WindGustDirNS"][1][0]

X_train.head(10)


*Standardistaion / Gestion des null *


In [ ]:
#df2 = X_train2
# correlation
#corr_matrix = df2.corr()
#corr_matrix


In [ ]:
#df["Evaporation"].value_counts()

In [ ]:
#from pandas.plotting import scatter_matrix
#attributes = ["RainTomorrow","RainToday","WindGustDirWE","WindGustDirNS"]
#scatter_matrix(X_train[attributes], figsize=(12,8))

**Normalisation**

In [ ]:
# separate X and y
y_train = X_train['RainTomorrow']
X_train = X_train.drop('RainTomorrow', axis=1)

In [ ]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy="median")
imputer.fit(X_train)
X = imputer.transform(X_train)
X_train2 = pd.DataFrame(X, columns=X_train.columns)
X_train2.head(5)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train_std=sc.fit_transform(X_train2)
X_train2 = pd.DataFrame(X_train_std, columns=X_train2.columns)
X_train2.head(5)


**Jeu de test, score**


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score


def alterDataSet(df):
    removeUnusedVariable(df)
    processDate(df)
    df = changeAllColumnWinDir(df)
    df["RainToday"] = df["RainToday"].map({'No': 0, 'Yes': 1})
    X_bis = imputer.transform(df)
    X2 = pd.DataFrame(X_bis, columns=df.columns)

    X_std=sc.transform(X2)
    X2 = pd.DataFrame(X_std, columns=df.columns)
    
    return X2

# separate X and y
X_test = test_set.drop('RainTomorrow', axis=1)
y_test = test_set['RainTomorrow']
y_test = y_test.map({'No': 0, 'Yes': 1})
# apply change on X_test
X_test2 = alterDataSet(X_test)
X_test2.shape


**SGDClassifier **



In [ ]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(random_state=42)
model.fit(X_train2, y_train)

res = model.predict(X_test2)

In [ ]:
#
#confusion_matrix(y_test, res)
#
print(f1_score(y_test, res))
print(accuracy_score(y_test, res))


# Sans les mois : f1=0.5604687356953219   accuracy= 0.8311825310313302
# Avec les mois : f1=0.5570107152669658   accuracy= 0.8299166637364183
# Avec les mois (distance par rapport à mi-année): f1=0.5735630163473369  accuracy= 0.829389219030205
# ==> Ne pas utiliser la date comme data


**Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train2,y_train)
y_pred = model_dt.predict(X_test2)
score = accuracy_score(y_test,y_pred)
print(score)

In [ ]:
#result = sklearn.tree.export_graphviz(model_dt, out_file=None)                

In [ ]:
print(os.listdir("..")) #tree.dot
#from graphviz import Digraph
print(model_dt)
#import graphviz

#with open("tree_1.dot") as f:
    #dot_graph = f.read()

# remove the display(...)

#graphviz.Source(result)

